In [1]:
import numpy as np
from tensorflow import keras
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense

In [3]:
model = Sequential()
model.add(SimpleRNN(4, use_bias=False, input_shape=(2,2)))
model.add(Dense(3, use_bias=False, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 4)                 24        
                                                                 
 dense (Dense)               (None, 3)                 12        
                                                                 
Total params: 36
Trainable params: 36
Non-trainable params: 0
_________________________________________________________________


In [4]:
import pandas as pd

In [5]:
column_names = ['w1', 'w2', 'w3']
trigrams = pd.read_csv(r'/home/sudarsun/projects/ML_DL_py_TF/Chapter12_RNN_LSTM_V3/Datasets/3Gram_love_data.txt', 
                       delimiter='\t',
                       names=column_names)
print('shape of the data', trigrams.shape)
print('random sample:\n', trigrams.sample(10))

shape of the data (5351, 3)
random sample:
         w1   w2    w3
2412  love   to  have
2897  love   to   see
2752  love   to   see
2470  love  the   way
3108  love   to   see
3496  love   to   see
4075  love   it  when
2655  love  the   way
1603  love   to  talk
1259  love   in    my


In [9]:
unique_words = []
for i in list(trigrams.columns.values):
    for j in pd.unique(trigrams[i]):
        unique_words.append(j)
unique_words = np.unique(unique_words)

print('count of unique words:', len(unique_words))
print('unique word list:', unique_words)

count of unique words: 139
unique word list: ['a' 'able' 'about' 'admit' 'affair' 'affection' 'all' 'and' 'another'
 'answer' 'as' 'at' 'be' 'because' 'being' 'better' 'between' 'bother'
 'break' 'care' 'cared' 'come' 'concern' 'country' 'cut' 'disappoint' 'do'
 'each' 'every' 'fact' 'feel' 'feeling' 'find' 'first' 'for' 'from' 'get'
 'go' 'god' 'going' 'got' 'hate' 'hated' 'hates' 'have' 'he' 'hear'
 'hearing' 'her' 'here' 'him' 'his' 'husband' 'i' 'idea' 'if' 'in'
 'interrupt' 'is' 'it' 'kind' 'know' 'leave' 'letter' 'life' 'like'
 'listen' 'look' 'lost' 'lot' 'love' 'loved' 'lovely' 'loves' 'loving'
 'make' 'makes' 'man' 'marriage' 'me' 'minute' 'more' 'most' 'much'
 'music' 'my' 'nature' 'neighbor' 'not' 'nothing' 'of' 'on' 'one' 'ones'
 'or' 'other' 'over' 'play' 'respect' 'say' 'see' 'sit' 'smell' 'so'
 'someone' 'song' 'sound' 'story' 'stronger' 'support' 'take' 'talk'
 'tell' 'than' 'that' 'the' 'them' 'they' 'think' 'this' 'thought' 'thy'
 'to' 'too' 'united' 'use' 'very' 'vie

In [10]:
word_indices = dict((w, i) for i, w in enumerate(unique_words))
indices_words = dict((i, w) for i, w in enumerate(unique_words))

print("word_indices dictionary\n", word_indices)
print("word_indices.keys\n", word_indices.keys())
print("word_indices.values\n", word_indices.values())
print("\n" + "#"*50)
print("indices_words dictionary\n", indices_words)
print("indices_words.keys\n", indices_words.keys())
print("indices_words.values\n", indices_words.values())

word_indices dictionary
 {'a': 0, 'able': 1, 'about': 2, 'admit': 3, 'affair': 4, 'affection': 5, 'all': 6, 'and': 7, 'another': 8, 'answer': 9, 'as': 10, 'at': 11, 'be': 12, 'because': 13, 'being': 14, 'better': 15, 'between': 16, 'bother': 17, 'break': 18, 'care': 19, 'cared': 20, 'come': 21, 'concern': 22, 'country': 23, 'cut': 24, 'disappoint': 25, 'do': 26, 'each': 27, 'every': 28, 'fact': 29, 'feel': 30, 'feeling': 31, 'find': 32, 'first': 33, 'for': 34, 'from': 35, 'get': 36, 'go': 37, 'god': 38, 'going': 39, 'got': 40, 'hate': 41, 'hated': 42, 'hates': 43, 'have': 44, 'he': 45, 'hear': 46, 'hearing': 47, 'her': 48, 'here': 49, 'him': 50, 'his': 51, 'husband': 52, 'i': 53, 'idea': 54, 'if': 55, 'in': 56, 'interrupt': 57, 'is': 58, 'it': 59, 'kind': 60, 'know': 61, 'leave': 62, 'letter': 63, 'life': 64, 'like': 65, 'listen': 66, 'look': 67, 'lost': 68, 'lot': 69, 'love': 70, 'loved': 71, 'lovely': 72, 'loves': 73, 'loving': 74, 'make': 75, 'makes': 76, 'man': 77, 'marriage': 78, 

In [11]:
w1_w2 = trigrams[['w1','w2']]
for i in list(w1_w2.columns.values):
    w1_w2[i] = w1_w2[i].map(word_indices)
w1_w2 = np.array(w1_w2)
w1_w2 = np.reshape(w1_w2, (w1_w2.shape[0], 2, 1))
w1_w2_hot = keras.utils.to_categorical(np.array(w1_w2), num_classes=len(word_indices))
print("word1_word2_onehot shape", w1_w2_hot.shape)

word1_word2_onehot shape (5351, 2, 139)


<ipython-input-11-b909edb95ee2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  w1_w2[i] = w1_w2[i].map(word_indices)


In [13]:
w3 = trigrams['w3'].map(word_indices)
w3_hot = keras.utils.to_categorical(np.array(w3), num_classes=len(word_indices))
print("word3_onehot shape is", w3_hot.shape)

word3_onehot shape is (5351, 139)


In [14]:
print("time steps", w1_w2_hot.shape[1])
print("input nodes", w1_w2_hot.shape[2])
print("output nodes", w3_hot.shape[1])

time steps 2
input nodes 139
output nodes 139


In [15]:
model_rnn = Sequential()
model_rnn.add(SimpleRNN(30, input_shape=(w1_w2_hot.shape[1], w1_w2_hot.shape[2])))
model_rnn.add(Dense(w3_hot.shape[1], activation='softmax'))
model_rnn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_1 (SimpleRNN)    (None, 30)                5100      
                                                                 
 dense_1 (Dense)             (None, 139)               4309      
                                                                 
Total params: 9,409
Trainable params: 9,409
Non-trainable params: 0
_________________________________________________________________


In [16]:
model_rnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_rnn.fit(w1_w2_hot, w3_hot, epochs=20)

Epoch 1/20
168/168 [==============================] - 2s 3ms/step - loss: 3.7629 - accuracy: 0.3962
Epoch 2/20
168/168 [==============================] - 0s 3ms/step - loss: 2.7107 - accuracy: 0.4894
Epoch 3/20
168/168 [==============================] - 0s 3ms/step - loss: 2.2918 - accuracy: 0.5349
Epoch 4/20
168/168 [==============================] - 0s 3ms/step - loss: 2.0187 - accuracy: 0.5771
Epoch 5/20
168/168 [==============================] - 0s 3ms/step - loss: 1.8363 - accuracy: 0.5960
Epoch 6/20
168/168 [==============================] - 0s 3ms/step - loss: 1.7030 - accuracy: 0.6137
Epoch 7/20
168/168 [==============================] - 0s 3ms/step - loss: 1.6005 - accuracy: 0.6287
Epoch 8/20
168/168 [==============================] - 0s 3ms/step - loss: 1.5165 - accuracy: 0.6384
Epoch 9/20
168/168 [==============================] - 0s 3ms/step - loss: 1.4476 - accuracy: 0.6472
Epoch 10/20
168/168 [==============================] - 0s 3ms/step - loss: 1.3922 - accuracy: 0.6563

In [26]:
def rnn_word_pred(in_text):
    print("input is", in_text)
    encoded = [word_indices[i] for i in in_text]
    encoded = np.array(encoded).reshape(1,2,1)
    encoded = keras.utils.to_categorical(np.array(encoded), num_classes=len(word_indices))
    yhat = np.argmax(model_rnn.predict(encoded, verbose=0))
    print("output -->", indices_words[yhat])

In [30]:
rnn_word_pred(['hate','me'])

input is ['hate', 'me']
output --> so
